In [2]:
import numpy as np
from phase_est_smc import phase_est_smc
import matplotlib.pyplot as plt
#import seaborn as sns; sns.set()

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
num_particles = 1000 # number of SMC particles (num of w points)
num_samples = 10000 # number of samples to draw from the particle distribution (to be binned)
num_bins = 100 # number of bins
n_runs = 100 # number of different omega*
t0 = 10 # starting time
max_iters = 100 # maximum number of iterations before breaking



In [ ]:
true_omegas = []
kde_preds = []
kde_data = []
rng = np.random.default_rng(200)

for i in range(n_runs): 

    true_omega = np.random.uniform(low=-1, high =1) * np.pi

    true_omegas.append(true_omega)
    smc = phase_est_smc(true_omega, t0, max_iters)
    smc.init_particles(num_particles)
    resample_counts = 0

    while True:

        particle_pos, particle_wgts = smc.particles(threshold=num_particles/5, num_measurements=1)
        if smc.break_flag:
            break
        smc.kde_resample(num_samples=num_samples, method=1, kernel='gaussian')
        resample_counts += 1

    kde_data.append(smc.data)
    kde_preds.append(smc.curr_omega_est)

    if np.abs(true_omega - smc.curr_omega_est) > 1:
        print("True omega: {:f}, prediction by KDE: {:f}, num of resample calls: {:d}. Failed".format(true_omega,
                                                                                            smc.curr_omega_est,
                                                                                            resample_counts))
    else:
        print("True omega: {:f}, prediction by KDE: {:f}, num of resample calls: {:d}".format(true_omega,
                                                                                            smc.curr_omega_est,
                                                                                            resample_counts))


In [ ]:
for i in range(len(smc.memory.pos_wgt_bef_res)-1):
    pos_bef_res = smc.memory.pos_wgt_bef_res[i][0]
    wgt_bef_res = smc.memory.pos_wgt_bef_res[i][1]
    pos_aft_res = smc.memory.pos_wgt_aft_res[i][0]
    wgt_aft_res = smc.memory.pos_wgt_aft_res[i][1]
    kernel = smc.memory.kernel[i]
    
    x_grid = np.linspace(pos_bef_res.min(), pos_bef_res.max(), 1000)
    prob = np.exp(kernel.score_samples(x_grid[:, None]))
    
    fig, (ax1, ax2) = plt.subplots(1,2, figsize=(12,6))
    ax1.hist(pos_bef_res, bins = num_bins, weights = wgt_bef_res, label='Before resampling')
    ax1.hist(pos_aft_res, bins = num_bins, weights = wgt_aft_res, label='After resampling', alpha=0.5)
    ax1.plot(true_omega, 0,marker='x', markersize=15)
    #ax1.set_xlim([-np.pi, np.pi])
    ax1.legend()
    
    ax2.fill_between(x_grid, prob, alpha=0.5)
    ax2.set_ylim(bottom=0)
    #ax2.set_xlim([-np.pi, np.pi])
    ax2.plot(true_omega, 0,marker='x', markersize=15)
    
    plt.show()
    

In [ ]:
kde_data_squared = ( np.array(kde_data) - np.array(true_omegas).reshape(-1,1)) ** 2 
kde_data_mean = np.mean(kde_data_squared, axis=0)
kde_data_median = np.median(kde_data_squared, axis =0)

print("MSE: {:f}, Median: {:f}".format(kde_data_mean[-1], kde_data_median[-1]))

In [ ]:
num_data_points = kde_data_squared.shape[1]

plt.plot(np.arange(1,num_data_points+1, dtype=int), kde_data_mean, label='Mean')
plt.plot(np.arange(1,num_data_points+1, dtype=int), kde_data_median, label='Median')
plt.legend()
plt.title("KDE")
plt.xlabel("Iters")
plt.ylabel("$(\omega - \omega*)^2$")
plt.yscale('log')
plt.show()